In [1]:
from collections import Counter
from itertools import product
import numpy as np
import pandas as pd
import pickle
import re
from scipy import spatial

In [2]:
vec_locs = sorted(["".join(x) for x in product("YMN", repeat=5)])
vec_locs[:5]

['MMMMM', 'MMMMN', 'MMMMY', 'MMMNM', 'MMMNN']

In [3]:
# Extract the Wordle result from a tweet, and remap the unicode feedback to Y/M/N
def wordle_guesses(tweet):
    text = (tweet.replace("Y", "y").replace("🟩", "Y")
                 .replace("M", "m").replace("🟨", "M")
                 .replace("N", "n").replace("⬛", "N").replace("⬜", "N"))
    guesses = re.findall("([YMN]+)", text)
    return guesses

def evaluate_guess_char(answer, guess, pos):
    if answer[pos]==guess[pos]:
        return "Y"
    unmatched_answer_chars = 0
    unmatched_guess_chars = 0
    this_guess_num = 0 
    for i in range(5):
        if answer[i]==guess[pos]:
            if answer[i]!=guess[i]:
                unmatched_answer_chars += 1
        if guess[i]==guess[pos]:
            if answer[i]!=guess[i]:
                unmatched_guess_chars += 1
                if i<pos:
                    this_guess_num += 1
    if this_guess_num<unmatched_answer_chars:
        return "M"
    return "N"

def evaluate_guess(answer, guess):
    return "".join(evaluate_guess_char(answer, guess, i) for i in range(5))

assert evaluate_guess("tools", "break")=="NNNNN"
assert evaluate_guess("tools", "tools")=="YYYYY"
assert evaluate_guess("tools", "tolls")=="YYNYY"
assert evaluate_guess("tools", "books")=="NYYNY"
assert evaluate_guess("tools", "broke")=="NNYNN"
assert evaluate_guess("tools", "yahoo")=="NNNMM"
assert evaluate_guess("tools", "bongo")=="NYNNM"
assert evaluate_guess("tools", "brook")=="NNYMN"
assert evaluate_guess("rates", "ranks")=="YYNNY"
assert evaluate_guess("rates", "apple")=="MNNNM"

In [ ]:
with open("../input/wordle-simulations/vec_all.pickle", "rb") as f:
    v_all = pickle.load(f)

with open("../input/wordle-simulations/vec_ratio.pickle", "rb") as f:
    v_ratio = pickle.load(f)

with open("../input/wordle-simulations/invalid_results.pickle", "rb") as f:
    invalid_results = pickle.load(f)
    
with open("../input/wordle-simulations/answers.txt", "r") as f:
    answers = f.read().split("\n")

with open("../input/wordle-simulations/other_words.txt", "r") as f:
    other_words = f.read().split("\n")

words = answers + other_words
for answer in ["agora", "pupal", "lynch", "harry", "fibre", "slave", "hasty", "fella"]: # NYTimes version skipped these
    answers.remove(answer)
    
# NYTimes version replaced these
answers[324] = "shiny"
answers[325] = "gecko"